<a href="https://colab.research.google.com/github/sumaiya08/medicaldata/blob/master/StatisticsofeICU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas.util.testing as tm
sns.set()

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
project_id='eicudata'

# Extracting dataset

In [0]:
%%bigquery --project $project_id patient
select gender,age,ethnicity,apacheadmissiondx,hospitaladmitsource,unittype,unitdischargestatus,hospitaldischargestatus from `physionet-data.eicu_crd.patient`;

## Removing missing values

In [0]:
missing = patient[(patient['gender'] == '') |(patient['gender'] == 'Other')|(patient['gender'] == 'Unknown')| (patient['ethnicity'] == '')|(patient['ethnicity'] == 'Other/Unknown')|(patient['unitdischargestatus']=='')|(patient['hospitaldischargestatus']=='')|(patient['age']=='')].index
patient = patient.drop(index=missing)
patient

,gender,age,ethnicity,apacheadmissiondx,hospitaladmitsource,unittype,unitdischargestatus,hospitaldischargestatus
1,Male,51,Caucasian,"Cardiovascular medical, other",Emergency Department,Med-Surg ICU,Alive,Alive
2,Female,31,Caucasian,Diabetic ketoacidosis,Emergency Department,Med-Surg ICU,Alive,Alive
3,Male,58,Caucasian,,Other Hospital,Med-Surg ICU,Alive,Alive
4,Female,68,Caucasian,,Other Hospital,Med-Surg ICU,Alive,Expired
5,Female,51,Caucasian,"ARDS-adult respiratory distress syndrome, non-...",Other Hospital,Med-Surg ICU,Alive,Expired
...,...,...,...,...,...,...,...,...
200854,Female,47,Caucasian,Coma/change in level of consciousness (for hep...,,Med-Surg ICU,Expired,Expired
200855,Female,46,Asian,Asthma,,Med-Surg ICU,Alive,Alive
200856,Male,66,Caucasian,"Pneumonia, bacterial",,Med-Surg ICU,Alive,Alive
200857,Male,69,Caucasian,GI obstruction,,Med-Surg ICU,Alive,Alive


In [0]:
patient['age'].replace({'> 89': '89'}, inplace=True)
patient['age']=patient['age'].apply(pd.to_numeric)

### Finding mean values, Note - age is the only numerical column here

In [0]:
patient.mean()

Series([], dtype: float64)

## Descriptive statistics of all columns

In [0]:
patient.describe(include='all')

,gender,age,ethnicity,apacheadmissiondx,hospitaladmitsource,unittype,unitdischargestatus,hospitaldischargestatus
count,187378,187378.000000,187378,187378,187378,187378,187378,187378
unique,2,NaN,5,392,16,8,2,2
top,Male,NaN,Caucasian,,Emergency Department,Med-Surg ICU,Alive,Alive
freq,100860,NaN,153916,21491,77097,104836,177192,170403
mean,NaN,63.232146,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,17.038186,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,53.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,65.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,76.000000,NaN,NaN,NaN,NaN,NaN,NaN


## Statistics of the data grouped by gender and ethnicity

In [0]:
patient.groupby(['gender','ethnicity']).describe()

age                        ...                  
                           count       mean        std  ...   50%   75%   max
gender ethnicity                                        ...                  
Female African American  10321.0  57.596454  17.551641  ...  59.0  70.0  89.0
       Asian              1541.0  63.525633  18.459331  ...  67.0  79.0  89.0
       Caucasian         70393.0  65.044834  17.236981  ...  68.0  79.0  89.0
       Hispanic           3423.0  62.046450  19.659415  ...  65.0  79.0  89.0
       Native American     840.0  53.132143  16.939784  ...  54.0  65.0  89.0
Male   African American  10806.0  56.060152  16.419762  ...  57.0  67.0  89.0
       Asian              1680.0  63.135714  16.918666  ...  65.0  77.0  89.0
       Caucasian         83523.0  63.765011  16.143166  ...  66.0  76.0  89.0
       Hispanic           4003.0  59.629278  18.891520  ...  61.0  75.0  89.0
       Native American     848.0  51.715802  16.554176  ...  53.0  64.0  89.0

[10 rows x 8 columns]

## Finding 95th percentile for column age

In [0]:
group1 = patient['age'].groupby(patient['gender']).quantile(0.95)
group1 = pd.DataFrame(group1)
group1
#patient.age.quantile(0.95)

,age
gender,
Female,89.0
Male,87.0


In [0]:
grouped_single = patient.groupby(['gender','ethnicity']).agg({'age': ['count','mean','std','min','max']})
grouped_single

age                              
                         count       mean        std min max
gender ethnicity                                            
Female African American  10321  57.596454  17.551641   1  89
       Asian              1541  63.525633  18.459331  16  89
       Caucasian         70393  65.044834  17.236981   0  89
       Hispanic           3423  62.046450  19.659415   0  89
       Native American     840  53.132143  16.939784  16  89
Male   African American  10806  56.060152  16.419762   1  89
       Asian              1680  63.135714  16.918666  14  89
       Caucasian         83523  63.765011  16.143166   0  89
       Hispanic           4003  59.629278  18.891520   0  89
       Native American     848  51.715802  16.554176  14  89

## Extracting features

In [0]:
%%bigquery --project $project_id variable
select temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph
from `physionet-data.eicu_crd.apacheapsvar` a
join `physionet-data.eicu_crd.patient` b ON (
a.patientunitstayid =b.patientunitstayid
)

In [0]:
variable

,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph
0,36.2,23.0,152,266.0,40.0,0.76,7.393
1,36.4,17.0,48,102.0,-1.0,1.04,-1.000
2,36.4,57.0,101,101.0,50.0,0.60,7.390
3,36.3,57.0,43,86.0,-1.0,0.75,-1.000
4,36.9,23.0,106,-1.0,-1.0,0.80,-1.000
...,...,...,...,...,...,...,...
171172,36.3,59.0,106,200.0,-1.0,-1.00,-1.000
171173,36.3,59.0,218,-1.0,-1.0,-1.00,-1.000
171174,36.6,59.0,56,61.0,-1.0,8.76,-1.000
171175,-1.0,59.0,169,174.0,-1.0,7.12,-1.000


## Statistics of features

In [0]:
variable.describe()

,temperature,respiratoryrate,heartrate,glucose,fio2,creatinine,ph
count,171177.000000,171177.000000,171177.000000,171177.000000,171177.000000,171177.000000,171177.000000
mean,33.728719,24.784735,98.667993,141.184455,12.794373,0.999311,0.906831
std,9.725657,15.301895,33.263814,109.723423,28.329034,1.852455,3.506569
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,36.100000,10.000000,84.000000,85.000000,-1.000000,0.470000,-1.000000
50%,36.400000,27.000000,104.000000,115.000000,-1.000000,0.810000,-1.000000
75%,36.700000,35.000000,120.000000,187.000000,-1.000000,1.360000,-1.000000
max,42.300000,60.000000,220.000000,2357.000000,100.000000,24.950000,7.810000


### 95% values

In [0]:
variable.quantile(0.95)

temperature         37.300
respiratoryrate     52.000
heartrate          147.000
glucose            333.000
fio2               100.000
creatinine           4.100
ph                   7.428
Name: 0.95, dtype: float64